# 1. Joining three Tables

In [2]:
import sqlite3
conn = sqlite3.connect("chinook.db")

join_q = '''
        SELECT 
            il.track_id,
            t.name track_name,
            mt.name track_type,
            il.unit_price,
            il.quantity
        FROM invoice_line il
        INNER JOIN track t ON t.track_id = il.track_id
        INNER JOIN media_type mt ON mt.media_type_id = t.media_type_id
        WHERE il.invoice_id = 4;
'''
joined_tables = conn.execute(join_q).fetchall()
print(joined_tables)

[(3448, 'Lamentations of Jeremiah, First Set \\ Incipit Lamentatio', 'Protected AAC audio file', 0.99, 1), (2560, 'Violent Pornography', 'MPEG audio file', 0.99, 1), (3336, 'War Pigs', 'Purchased AAC audio file', 0.99, 1), (829, "Let's Get Rocked", 'MPEG audio file', 0.99, 1), (1872, 'Attitude', 'MPEG audio file', 0.99, 1), (748, 'Dealer', 'MPEG audio file', 0.99, 1), (1778, "You're What's Happening (In The World Today)", 'MPEG audio file', 0.99, 1), (2514, 'Spoonman', 'MPEG audio file', 0.99, 1)]


# 2. Joining more than 3

In [4]:
join_4 = ''' 
        SELECT
            il.track_id,
            t.name track_name,
            ar.name artist_name,
            mt.name track_type,
            il.unit_price,
            il.quantity
        FROM invoice_line il
        INNER JOIN track t ON t.track_id = il.track_id
        INNER JOIN media_type mt ON mt.media_type_id = t.media_type_id
        INNER JOIN album al ON al.album_id = t.album_id
        INNER JOIN artist ar on ar.artist_id = al.artist_id
        WHERE il.invoice_id = 4;
'''
joined_4 = conn.execute(join_4).fetchall()
print(joined_4)

[(3448, 'Lamentations of Jeremiah, First Set \\ Incipit Lamentatio', "The King's Singers", 'Protected AAC audio file', 0.99, 1), (2560, 'Violent Pornography', 'System Of A Down', 'MPEG audio file', 0.99, 1), (3336, 'War Pigs', 'Cake', 'Purchased AAC audio file', 0.99, 1), (829, "Let's Get Rocked", 'Def Leppard', 'MPEG audio file', 0.99, 1), (1872, 'Attitude', 'Metallica', 'MPEG audio file', 0.99, 1), (748, 'Dealer', 'Deep Purple', 'MPEG audio file', 0.99, 1), (1778, "You're What's Happening (In The World Today)", 'Marvin Gaye', 'MPEG audio file', 0.99, 1), (2514, 'Spoonman', 'Soundgarden', 'MPEG audio file', 0.99, 1)]


# 3. joining with subqueries

In [5]:
join_sub = '''
        SELECT
            ta.album_title album,
            ta.artist_name artist,
            COUNT(*) tracks_purchased
        FROM invoice_line il
        INNER JOIN (
                    SELECT
                        t.track_id,
                        al.title album_title,
                        ar.name artist_name
                    FROM track t
                    INNER JOIN album al ON al.album_id = t.album_id
                    INNER JOIN artist ar ON ar.artist_id = al.artist_id
                   ) ta
                   ON ta.track_id = il.track_id
        GROUP BY 1, 2
        ORDER BY 3 DESC LIMIT 5;
'''
joined_sub = conn.execute(join_4).fetchall()
print(joined_sub)


[(3448, 'Lamentations of Jeremiah, First Set \\ Incipit Lamentatio', "The King's Singers", 'Protected AAC audio file', 0.99, 1), (2560, 'Violent Pornography', 'System Of A Down', 'MPEG audio file', 0.99, 1), (3336, 'War Pigs', 'Cake', 'Purchased AAC audio file', 0.99, 1), (829, "Let's Get Rocked", 'Def Leppard', 'MPEG audio file', 0.99, 1), (1872, 'Attitude', 'Metallica', 'MPEG audio file', 0.99, 1), (748, 'Dealer', 'Deep Purple', 'MPEG audio file', 0.99, 1), (1778, "You're What's Happening (In The World Today)", 'Marvin Gaye', 'MPEG audio file', 0.99, 1), (2514, 'Spoonman', 'Soundgarden', 'MPEG audio file', 0.99, 1)]


# 4. Recursive joins

In [6]:
rec_q = '''
        SELECT
            e1.first_name || " " || e1.last_name employee_name,
            e1.title employee_title,
            e2.first_name || " " || e2.last_name supervisor_name,
            e2.title supervisor_title
        FROM employee e1
        LEFT JOIN employee e2 ON e1.reports_to = e2.employee_id
        ORDER BY 1;
'''
rec_join = conn.execute(rec_q).fetchall()
print(rec_join)

[('Andrew Adams', 'General Manager', None, None), ('Jane Peacock', 'Sales Support Agent', 'Nancy Edwards', 'Sales Manager'), ('Laura Callahan', 'IT Staff', 'Michael Mitchell', 'IT Manager'), ('Margaret Park', 'Sales Support Agent', 'Nancy Edwards', 'Sales Manager'), ('Michael Mitchell', 'IT Manager', 'Andrew Adams', 'General Manager'), ('Nancy Edwards', 'Sales Manager', 'Andrew Adams', 'General Manager'), ('Robert King', 'IT Staff', 'Michael Mitchell', 'IT Manager'), ('Steve Johnson', 'Sales Support Agent', 'Nancy Edwards', 'Sales Manager')]


# 5. Case

In [7]:
case_q = '''
        SELECT
           c.first_name || " " || c.last_name customer_name,
           COUNT(i.invoice_id) number_of_purchases,
           SUM(i.total) total_spent,
           CASE
               WHEN sum(i.total) < 40 THEN 'small spender'
               WHEN sum(i.total) > 100 THEN 'big spender'
               ELSE 'regular'
               END
               AS customer_category
        FROM invoice i
        INNER JOIN customer c ON i.customer_id = c.customer_id
        GROUP BY 1 ORDER BY 1;
'''
case_join = conn.execute(case_q).fetchall()
print(case_join)

[('Aaron Mitchell', 8, 70.28999999999999, 'regular'), ('Alexandre Rocha', 10, 69.3, 'regular'), ('Astrid Gruber', 9, 69.3, 'regular'), ('Bjørn Hansen', 9, 72.27000000000001, 'regular'), ('Camille Bernard', 9, 79.2, 'regular'), ('Daan Peeters', 7, 60.38999999999999, 'regular'), ('Dan Miller', 12, 95.03999999999999, 'regular'), ('Diego Gutiérrez', 5, 39.6, 'small spender'), ('Dominique Lefebvre', 9, 72.27, 'regular'), ('Eduardo Martins', 12, 60.39, 'regular'), ('Edward Francis', 13, 91.08, 'regular'), ('Ellie Sullivan', 12, 75.24000000000001, 'regular'), ('Emma Jones', 8, 68.31, 'regular'), ('Enrique Muñoz', 11, 98.01, 'regular'), ('Fernanda Ramos', 15, 106.91999999999999, 'big spender'), ('Frank Harris', 8, 74.25, 'regular'), ('Frank Ralston', 8, 71.28, 'regular'), ('František Wichterlová', 18, 144.54000000000002, 'big spender'), ('François Tremblay', 9, 99.99, 'regular'), ('Fynn Zimmermann', 10, 94.05000000000001, 'regular'), ('Hannah Schneider', 11, 85.14, 'regular'), ('Heather Leacoc